In [41]:
import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from textblob import TextBlob
import re

In [42]:
def remove_mentions(tweet):
    return re.sub('@[A-Za-z0-9_]+'," ",tweet)

def remove_hashtags(tweet): 
    return re.sub('#[A-Za-z0-9_]+'," ",tweet)

def remove_links(tweet): 
    tweet = re.sub('[()!?]'," ",tweet)           
    tweet = re.sub(r'http\S+'," ",tweet)         
    tweet = re.sub(r'www.\S+'," ",tweet)         
    tweet = re.sub(r'\[.*?\]',' ', tweet)        
    return re.sub(r'[^a-z0-9]'," ",tweet)         

def get_sentiment(tweet):
    blob = TextBlob(tweet)
    return blob.sentiment.polarity

def get_sentiment_name(sentiment):
    if sentiment == 0:
        return "Neutral 😊"
    elif sentiment > 0:
        return "Positive🥳"
    else:
        return "Negative🤬"

def load_data():
    df = pd.read_csv('Valorant.csv')
    df.rename(columns={'Text':'Tweets'}, inplace = True)
    df['Tweets'] = df['Tweets'].str.lower()
    df['Tweets'] = df['Tweets'].apply(remove_hashtags)
    df['Tweets'] = df['Tweets'].apply(remove_mentions)
    df['Tweets'] = df['Tweets'].apply(remove_links)
    df['Sentiment'] = df['Tweets'].apply(get_sentiment)
    df['SentimentName'] = df["Sentiment"].apply(get_sentiment_name)
    return df

In [43]:
df = load_data()

In [44]:
counter = df.SentimentName.reset_index()
c1,c2 = st.columns([1,3])
# counter.value_counts()
counter['Date'] = df['Date']
# counter

In [45]:
counter[counter['Date']<='2020-02-04']

,index,SentimentName,Date
0,0,Neutral 😊,02-03-2020 07:57
1,1,Neutral 😊,02-03-2020 08:01
2,2,Neutral 😊,02-03-2020 20:40
3,3,Positive🥳,02-03-2020 20:43
4,4,Negative🤬,02-03-2020 20:45
...,...,...,...
327,327,Positive🥳,20-04-2020 22:04
328,328,Positive🥳,20-04-2020 22:06
329,329,Positive🥳,20-04-2020 23:21
330,330,Neutral 😊,20-04-2020 23:30


In [46]:
counter = pd.concat([counter,pd.get_dummies(counter['SentimentName'])], axis=1)

In [47]:
counter['Date'] = pd.to_datetime(counter['Date'])
counterdf = counter.set_index('Date').resample('D').sum()
counterdf.replace(0,np.nan,inplace=True)

In [48]:
counterdf.dropna(how='all',inplace=True)

In [49]:
px.funnel(counterdf, counterdf.index, ['Negative🤬','Neutral 😊','Positive🥳'])